# Chehra: FACECOM Face Analysis in Challenging Environments (COMSYS 5 Hackathon Submission) - Task A: Gender Classification

Welcome to "**Chehra**," our submission for the **COMSYS 5 FACECOM Hackathon**. This notebook focuses *primarily* on **Task A: Gender Classification (Binary Classification)**, addressing the challenge of predicting gender from face images under various visual degradations such as motion blur, fog, rain, low light, and uneven lighting.

The overall hackathon includes two main tasks:

- **Task A: Gender Classification (Binary Classification)**
- **Task B: Face Recognition (Multi-class Classification)**

The dataset used is **FACECOM (Face Attributes in Challenging Environments)**, specifically designed for this challenge with diverse visual conditions and annotations for Gender and Person Identity.

This notebook is organized to present our solutions for Task A, including training, validation, and ensemble inference. While Task B is part of the hackathon, this notebook focuses on Task A.

Let's dive into the details of our approach for Task A.

# Task A: Gender Classification (Binary Classification)

This section focuses on **Task A**: **Gender Classification**. The goal is to build a robust model that can accurately predict the gender (Male/Female) from face images, even those affected by various visual degradations.

We are using a **ResNet18** model, pre-trained on ImageNet, and fine-tuning it on the provided Task A training dataset. ResNet models are chosen for their balance of performance and computational efficiency, as well as their effectiveness in handling edge cases and distortions, making them suitable for this challenging dataset.

Key techniques used in the training process include:
- **Albumentations** for extensive data augmentation to improve robustness to variations in the data, including various distortions like blur, fog, noise, and rain.
- **Weighted Random Sampler** to address class imbalance in the training data by oversampling the minority class.
- **Class Weights** in the loss function to further mitigate the impact of class imbalance.

The model's performance is evaluated using **Accuracy**, **Precision**, **Recall**, and **F1-Score** on the validation set.

This script contains the core training logic for **Task A: Gender Classification**. It utilizes a ResNet18 model, applies extensive data augmentation using Albumentations, handles class imbalance with Weighted Random Sampler and class weights in the loss function, and trains the model for a specified number of epochs, saving the best model based on training F1-Score.

In [ ]:
import os
import argparse
import random
import numpy as np
from tqdm import tqdm
from PIL import Image
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import models
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

import albumentations as A
from albumentations.pytorch import ToTensorV2

# ----------------------- Dataset Class ----------------------- #
class TaskAGenderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        self.label_map = {"male": 0, "female": 1}
        self.label_counts = {0: 0, 1: 0}  # Track counts for weighting

        for label_name in ["male", "female"]:
            folder = root_dir / label_name
            if not folder.exists():
                print(f"⚠️ Folder not found: {folder}")
                continue

            for ext in ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]:
                for img_path in folder.glob(ext):
                    label = self.label_map[label_name]
                    self.samples.append((img_path, label))
                    self.label_counts[label] += 1

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert("RGB")
        image = np.array(image)
        if self.transform:
            image = self.transform(image=image)["image"]
        return image, label

# ----------------------- Training Function ----------------------- #
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []

    for images, labels in tqdm(loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    return total_loss / len(loader.dataset), acc, precision, recall, f1

# ----------------------- Reproducibility ----------------------- #
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# ----------------------- Main Function ----------------------- #
def main(args):
    set_seed()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_transform = A.Compose([
        A.Resize(256, 256),
        A.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.33)),
        A.HorizontalFlip(p=0.5),

        A.OneOf([
            A.GaussianBlur(blur_limit=(3, 7), p=0.5),
            A.MotionBlur(blur_limit=5, p=0.5),
        ], p=0.3),

        A.OneOf([
            A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.6, alpha_coef=0.1, p=1.0),
            A.RandomBrightnessContrast(brightness_limit=(-0.6, -0.3), contrast_limit=0.2, p=1.0),
            A.RandomBrightnessContrast(brightness_limit=(0.5, 0.9), contrast_limit=0.2, p=1.0),
        ], p=0.4),

        A.OneOf([
            A.GaussNoise(var_limit=(20.0, 60.0), p=0.5),
            A.ISONoise(intensity=(0.2, 0.5), p=0.5),
            A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=0.5),
        ], p=0.3),

        A.OneOf([
            A.Downscale(scale_min=0.3, scale_max=0.7, interpolation=1, p=1.0),
            A.ImageCompression(quality_lower=10, quality_upper=30, p=1.0),
        ], p=0.3),

        A.RandomRain(blur_value=2, brightness_coefficient=0.9, drop_color=(200, 200, 200), p=0.3),

        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

    train_data = TaskAGenderDataset(Path(args.train_path), transform=train_transform)

    # --- Compute class weights --- #
    total = train_data.label_counts[0] + train_data.label_counts[1]
    male_weight = total / (2 * train_data.label_counts[0])
    female_weight = total / (2 * train_data.label_counts[1])
    class_weights = torch.tensor([male_weight, female_weight], dtype=torch.float32).to(device)
    print(f"Class weights: {class_weights.tolist()}")

    # Optional: Oversample females
    targets = [label for _, label in train_data.samples]
    class_sample_count = np.bincount(targets)
    weights = 1. / class_sample_count[targets]
    sampler = WeightedRandomSampler(weights, len(weights), replacement=True)

    train_loader = DataLoader(train_data, batch_size=args.batch_size, sampler=sampler)

    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 2)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.AdamW(model.parameters(), lr=args.lr)

    best_f1 = 0.0
    for epoch in range(args.epochs):
        print(f"\nEpoch {epoch+1}/{args.epochs}")

        train_loss, train_acc, train_prec, train_rec, train_f1 = train_one_epoch(
            model, train_loader, optimizer, criterion, device
        )

        print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, Prec: {train_prec:.4f}, Rec: {train_rec:.4f}, F1: {train_f1:.4f}")

        if train_f1 > best_f1:
            best_f1 = train_f1
            torch.save(model.state_dict(), args.save_path)
            full_model_path = args.save_path.replace('.pth', '_full.pt')
            torch.save(model, full_model_path)
            print(f"\n✅ Best model saved with F1: {best_f1:.4f}")

if __name__ == "__main__":
    class Args:
        train_path = "/kaggle/input/comsys/Comys_Hackathon5/Task_A/train"
        batch_size = 64
        epochs = 25
        lr = 1e-4
        save_path = "best_gender_model.pth"

    args = Args()

    temp_dataset = TaskAGenderDataset(Path(args.train_path))
    if len(temp_dataset) == 0:
        raise ValueError(f"No images found in {args.train_path}. Check if 'male' and 'female' folders contain images.")

    main(args)


/tmp/ipykernel_35/3613758861.py:96: UserWarning: Argument(s) 'fog_coef_lower, fog_coef_upper' are not valid for transform RandomFog
  A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.6, alpha_coef=0.1, p=1.0),
/tmp/ipykernel_35/3613758861.py:102: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(20.0, 60.0), p=0.5),
/tmp/ipykernel_35/3613758861.py:108: UserWarning: Argument(s) 'scale_min, scale_max, interpolation' are not valid for transform Downscale
  A.Downscale(scale_min=0.3, scale_max=0.7, interpolation=1, p=1.0),
/tmp/ipykernel_35/3613758861.py:109: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=10, quality_upper=30, p=1.0),
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Class weights: [0.6285901069641113, 2.444162368774414]

Epoch 1/25
Training: 100%|██████████| 31/31 [00:27<00:00,  1.14it/s]
Train - Loss: 0.3338, Acc: 0.7342, Prec: 0.6681, Rec: 0.9669, F1: 0.7902

✅ Best model saved with F1: 0.7902

Epoch 2/25
Training: 100%|██████████| 31/31 [00:30<00:00,  1.03it/s]
Train - Loss: 0.2520, Acc: 0.7996, Prec: 0.7237, Rec: 0.9753, F1: 0.8309

✅ Best model saved with F1: 0.8309

Epoch 3/25
Training: 100%|██████████| 31/31 [00:31<00:00,  1.02s/it]
Train - Loss: 0.2118, Acc: 0.8401, Prec: 0.7654, Rec: 0.9634, F1: 0.8531

✅ Best model saved with F1: 0.8531

Epoch 4/25
Training: 100%|██████████| 31/31 [00:29<00:00,  1.05it/s]
Train - Loss: 0.1716, Acc: 0.8671, Prec: 0.7982, Rec: 0.9877, F1: 0.8829

✅ Best model saved with F1: 0.8829

Epoch 5/25
Training: 100%|██████████| 31/31 [00:28<00:00,  1.10it/s]
Train - Loss: 0.1516, Acc: 0.8884, Prec: 0.8360, Rec: 0.9668, F1: 0.8967

✅ Best model saved with F1: 0.8967

Epoch 6/25
Training: 100%|██████████| 31/31 [00:28<00:00,  1.08it/s]
Train - Loss: 0.1337, Acc: 0.9003, Prec: 0.8477, Rec: 0.9773, F1: 0.9079

✅ Best model saved with F1: 0.9079

Epoch 7/25
Training: 100%|██████████| 31/31 [00:26<00:00,  1.18it/s]
Train - Loss: 0.1295, Acc: 0.9045, Prec: 0.8506, Rec: 0.9824, F1: 0.9118

✅ Best model saved with F1: 0.9118

Epoch 8/25
Training: 100%|██████████| 31/31 [00:27<00:00,  1.11it/s]
Train - Loss: 0.1415, Acc: 0.8956, Prec: 0.8432, Rec: 0.9765, F1: 0.9050

Epoch 9/25
Training: 100%|██████████| 31/31 [00:25<00:00,  1.22it/s]
Train - Loss: 0.1356, Acc: 0.8941, Prec: 0.8288, Rec: 0.9816, F1: 0.8987

Epoch 10/25
Training: 100%|██████████| 31/31 [00:28<00:00,  1.10it/s]
Train - Loss: 0.0973, Acc: 0.9351, Prec: 0.8971, Rec: 0.9869, F1: 0.9398

✅ Best model saved with F1: 0.9398

Epoch 11/25
Training: 100%|██████████| 31/31 [00:25<00:00,  1.19it/s]
Train - Loss: 0.1363, Acc: 0.9024, Prec: 0.8501, Rec: 0.9713, F1: 0.9067

Epoch 12/25
Training: 100%|██████████| 31/31 [00:26<00:00,  1.16it/s]
Train - Loss: 0.1007, Acc: 0.9268, Prec: 0.8829, Rec: 0.9845, F1: 0.9309

Epoch 13/25
Training: 100%|██████████| 31/31 [00:25<00:00,  1.20it/s]
Train - Loss: 0.0981, Acc: 0.9299, Prec: 0.8858, Rec: 0.9898, F1: 0.9349

Epoch 14/25
Training: 100%|██████████| 31/31 [00:27<00:00,  1.12it/s]
Train - Loss: 0.1033, Acc: 0.9247, Prec: 0.8749, Rec: 0.9884, F1: 0.9282

Epoch 15/25
Training: 100%|██████████| 31/31 [00:26<00:00,  1.17it/s]
Train - Loss: 0.0797, Acc: 0.9429, Prec: 0.9007, Rec: 0.9948, F1: 0.9454

✅ Best model saved with F1: 0.9454

Epoch 16/25
Training: 100%|██████████| 31/31 [00:25<00:00,  1.20it/s]
Train - Loss: 0.0755, Acc: 0.9496, Prec: 0.9159, Rec: 0.9896, F1: 0.9513

✅ Best model saved with F1: 0.9513

Epoch 17/25
Training: 100%|██████████| 31/31 [00:26<00:00,  1.17it/s]
Train - Loss: 0.0685, Acc: 0.9465, Prec: 0.9129, Rec: 0.9909, F1: 0.9503

Epoch 18/25
Training: 100%|██████████| 31/31 [00:26<00:00,  1.16it/s]
Train - Loss: 0.0817, Acc: 0.9507, Prec: 0.9199, Rec: 0.9876, F1: 0.9525

✅ Best model saved with F1: 0.9525

Epoch 19/25
Training: 100%|██████████| 31/31 [00:25<00:00,  1.22it/s]
Train - Loss: 0.0723, Acc: 0.9486, Prec: 0.9153, Rec: 0.9862, F1: 0.9494

Epoch 20/25
Training: 100%|██████████| 31/31 [00:25<00:00,  1.20it/s]
Train - Loss: 0.0687, Acc: 0.9491, Prec: 0.9116, Rec: 0.9948, F1: 0.9514

Epoch 21/25
Training: 100%|██████████| 31/31 [00:26<00:00,  1.19it/s]
Train - Loss: 0.0773, Acc: 0.9444, Prec: 0.9128, Rec: 0.9858, F1: 0.9479

Epoch 22/25
Training: 100%|██████████| 31/31 [00:26<00:00,  1.17it/s]
Train - Loss: 0.0678, Acc: 0.9569, Prec: 0.9311, Rec: 0.9900, F1: 0.9596

✅ Best model saved with F1: 0.9596

Epoch 23/25
Training: 100%|██████████| 31/31 [00:26<00:00,  1.18it/s]
Train - Loss: 0.0716, Acc: 0.9512, Prec: 0.9185, Rec: 0.9896, F1: 0.9527

Epoch 24/25
Training: 100%|██████████| 31/31 [00:27<00:00,  1.11it/s]
Train - Loss: 0.0594, Acc: 0.9553, Prec: 0.9237, Rec: 0.9927, F1: 0.9570

Epoch 25/25
Training: 100%|██████████| 31/31 [00:26<00:00,  1.16it/s]
Train - Loss: 0.0773, Acc: 0.9502, Prec: 0.9212, Rec: 0.9856, F1: 0.9523


This script performs inference and evaluation of the trained Gender Classification model (Task A) on the official validation set (`/kaggle/input/comsys-hack-dataset/Comys_Hackathon5/Task_A/val`). It loads the best saved model, runs predictions on the validation data, and prints a detailed classification report including Accuracy, Precision, Recall, and F1-Score. It also saves the predictions to a CSV file.

In [ ]:
import os
import numpy as np
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import classification_report

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

# ----------- Dataset for Inference ----------- #
class GenderValDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        self.label_map = {"male": 0, "female": 1}

        for label_name in ["male", "female"]:
            folder = root_dir / label_name
            for ext in ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]:
                for img_path in folder.glob(ext):
                    self.samples.append((img_path, self.label_map[label_name]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label, path.name  # also return filename

# ----------- Transforms ----------- #
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ----------- Load Model ----------- #
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18()
model.fc = torch.nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("/kaggle/input/model-and-imagestst/best_gender_model best one.pth", map_location=device))
model.to(device)
model.eval()

# ----------- Load Validation Dataset ----------- #
val_dir = Path("/kaggle/input/comsys-hack-dataset/Comys_Hackathon5/Task_A/val")
val_dataset = GenderValDataset(val_dir, transform=val_transform)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# ----------- Inference ----------- #
all_preds, all_labels, filenames = [], [], []
with torch.no_grad():
    for images, labels, names in tqdm(val_loader, desc="Running Inference"):
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels.numpy())
        filenames.extend(names)

# ----------- Report ----------- #
print("\n📊 Classification Report on Validation Set:")
print(classification_report(all_labels, all_preds, target_names=["male", "female"]))

# ----------- Optional: Save predictions to CSV ----------- #
import pandas as pd
df = pd.DataFrame({
    "filename": filenames,
    "true_label": all_labels,
    "predicted_label": all_preds
})
df.to_csv("gender_predictions_val.csv", index=False)
print("\n✅ Predictions saved to gender_predictions_val.csv")


This script provides a custom inference function for the trained Gender Classification model (Task A) allowing predictions on a specified folder of images. It loads a saved model, preprocesses images, performs inference, and can optionally save predictions to a CSV and display a grid of images with their predicted labels and confidence scores.

In [ ]:
import os
import torch
import pandas as pd
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

from torchvision import models, transforms
from torch.nn import functional as F

# ✅ Image Preprocessing (same as training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ✅ Label Map
idx2label = {0: "male", 1: "female"}

# ✅ Inference Function
def run_inference(model_path, image_folder, show_visuals=True, save_csv=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model
    model = models.resnet18(pretrained=False)
    model.fc = torch.nn.Linear(model.fc.in_features, 2)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()

    image_folder = Path(image_folder)
    image_paths = sorted([p for p in image_folder.glob("*") if p.suffix.lower() in ['.jpg', '.jpeg', '.png']])

    if len(image_paths) == 0:
        raise ValueError(f"No image files found in {image_folder}")

    predictions = []
    images_for_display = []

    with torch.no_grad():
        for img_path in image_paths:
            image = Image.open(img_path).convert("RGB")
            input_tensor = transform(image).unsqueeze(0).to(device)

            output = model(input_tensor)
            probs = F.softmax(output, dim=1)
            pred_label = torch.argmax(probs, dim=1).item()
            pred_conf = probs[0][pred_label].item()

            predictions.append({
                "filename": img_path.name,
                "predicted_label": idx2label[pred_label],
                "confidence": round(pred_conf, 4)
            })

            # Save for visualization
            images_for_display.append((image.resize((128, 128)), idx2label[pred_label], round(pred_conf, 2)))

    # ✅ Save Predictions CSV
    df = pd.DataFrame(predictions)
    if save_csv:
        df.to_csv("custom_predictions.csv", index=False)
        print("✅ Predictions saved to custom_predictions.csv")

    # ✅ Display Sample Grid
    if show_visuals:
        show_predictions_grid(images_for_display)

    return df

# ✅ Display Grid of Predictions
def show_predictions_grid(images_with_labels):
    num_images = len(images_with_labels)
    cols = 5
    rows = (num_images + cols - 1) // cols
    plt.figure(figsize=(15, 3 * rows))

    for i in range(num_images):
        img, label, conf = images_with_labels[i]
        plt.subplot(rows, cols, i + 1)
        plt.imshow(img)
        plt.title(f"{label} ({conf*100:.1f}%)", fontsize=10)
        plt.axis("off")

    plt.tight_layout()
    plt.show()

# ✅ Example Usage
if __name__ == "__main__":
    # Change these paths:
    model_path = "/kaggle/input/model-and-imagestst/best_gender_model best one.pth"
    test_images_path = "/kaggle/input/model-and-imagestst"

    run_inference(model_path, test_images_path)


This code cell contains a training script for a second ResNet18 model for Task A (Gender Classification). Similar to the first training script, it includes the `TaskAGenderDataset` class, `train_one_epoch` function, reproducibility settings, and the main training loop. It utilizes Albumentations for data augmentation, WeightedRandomSampler and class weights for handling class imbalance, and saves the best model based on training F1-Score.

In [ ]:
#task A second model running code
import os
import argparse
import random
import numpy as np
from tqdm import tqdm
from PIL import Image
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import models
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

import albumentations as A
from albumentations.pytorch import ToTensorV2

# ----------------------- Dataset Class ----------------------- #
class TaskAGenderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        self.label_map = {"male": 0, "female": 1}
        self.label_counts = {0: 0, 1: 0}

        for label_name in ["male", "female"]:
            folder = root_dir / label_name
            if not folder.exists():
                print(f"⚠️ Folder not found: {folder}")
                continue

            for ext in ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]:
                for img_path in folder.glob(ext):
                    label = self.label_map[label_name]
                    self.samples.append((img_path, label))
                    self.label_counts[label] += 1

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert("RGB")
        image = np.array(image)
        if self.transform:
            image = self.transform(image=image)["image"]
        return image, label

# ----------------------- Training Function ----------------------- #
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []

    for images, labels in tqdm(loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    return total_loss / len(loader.dataset), acc, precision, recall, f1

# ----------------------- Reproducibility ----------------------- #
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# ----------------------- Main Function ----------------------- #
def main(args):
    set_seed()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_transform = A.Compose([
        A.Resize(256, 256),
        A.RandomResizedCrop(size=(224, 224), scale=(0.7, 1.0), ratio=(0.75, 1.33)),
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=15, p=0.4),

        A.OneOf([
            A.GaussianBlur(blur_limit=(3, 7), p=0.5),
            A.MotionBlur(blur_limit=5, p=0.5),
        ], p=0.3),

        A.OneOf([
            A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.6, alpha_coef=0.1),
            A.RandomBrightnessContrast(brightness_limit=(-0.6, -0.3), contrast_limit=0.2),
            A.RandomBrightnessContrast(brightness_limit=(0.5, 0.9), contrast_limit=0.2),
        ], p=0.4),

        A.OneOf([
            A.GaussNoise(var_limit=(20.0, 60.0), p=0.5),
            A.ISONoise(intensity=(0.2, 0.5), p=0.5),
            A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=0.5),
        ], p=0.3),

        A.OneOf([
            A.Downscale(scale_min=0.3, scale_max=0.7, interpolation=1),
            A.ImageCompression(quality_lower=10, quality_upper=30),
        ], p=0.3),

        A.RandomRain(blur_value=2, brightness_coefficient=0.9, drop_color=(200, 200, 200), p=0.3),

        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

    train_data = TaskAGenderDataset(Path(args.train_path), transform=train_transform)

    # --- Compute class weights --- #
    total = train_data.label_counts[0] + train_data.label_counts[1]
    male_weight = total / (2 * train_data.label_counts[0])
    female_weight = total / (2 * train_data.label_counts[1])
    class_weights = torch.tensor([male_weight, female_weight], dtype=torch.float32).to(device)
    print(f"Class weights: {class_weights.tolist()}")

    # Optional: Oversample minority
    targets = [label for _, label in train_data.samples]
    class_sample_count = np.bincount(targets)
    weights = 1. / class_sample_count[targets]
    sampler = WeightedRandomSampler(weights, len(weights), replacement=True)

    train_loader = DataLoader(train_data, batch_size=args.batch_size, sampler=sampler)

    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 2)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.AdamW(model.parameters(), lr=args.lr)

    best_f1 = 0.0
    for epoch in range(args.epochs):
        print(f"\nEpoch {epoch+1}/{args.epochs}")

        train_loss, train_acc, train_prec, train_rec, train_f1 = train_one_epoch(
            model, train_loader, optimizer, criterion, device
        )

        print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, Prec: {train_prec:.4f}, Rec: {train_rec:.4f}, F1: {train_f1:.4f}")

        if train_f1 > best_f1:
            best_f1 = train_f1
            torch.save(model.state_dict(), args.save_path)
            torch.save(model, args.save_path.replace('.pth', '_full.pt'))
            print(f"✅ Best model saved with F1: {best_f1:.4f}")

# ----------------------- Args ----------------------- #
if __name__ == "__main__":
    class Args:
        train_path = "/kaggle/input/comsys/Comys_Hackathon5/Task_A/train"
        batch_size = 64
        epochs = 25
        lr = 1e-4
        save_path = "best_gender_model.pth"

    args = Args()

    temp_dataset = TaskAGenderDataset(Path(args.train_path))
    if len(temp_dataset) == 0:
        raise ValueError(f"No images found in {args.train_path}. Check if 'male' and 'female' folders contain images.")

    main(args)

This code cell provides a validation script for evaluating the second trained Gender Classification model on the validation dataset. It includes label maps, image transforms, a function to load the model state dictionary, and the main evaluation logic that calculates and prints a detailed classification report and displays a confusion matrix.

In [ ]:
#task A second model val code
import os
import torch
import numpy as np
from pathlib import Path
from PIL import Image
from tqdm import tqdm

from torchvision import models, transforms
from torch.nn import functional as F
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# ---------------- Label Maps ---------------- #
idx2label = {0: "male", 1: "female"}
label2idx = {v: k for k, v in idx2label.items()}

# ---------------- Transforms ---------------- #
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ---------------- Load Model ---------------- #
def load_model(model_path):
    model = models.resnet18(pretrained=False)
    model.fc = torch.nn.Linear(model.fc.in_features, 2)
    model.load_state_dict(torch.load(model_path, map_location="cuda" if torch.cuda.is_available() else "cpu"))
    model.eval()
    return model.to("cuda" if torch.cuda.is_available() else "cpu")

# ---------------- Validation Script ---------------- #
def evaluate_on_validation(model_path, val_folder):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = load_model(model_path)
    val_folder = Path(val_folder)

    image_paths = []
    true_labels = []
    pred_labels = []

    for label in ["male", "female"]:
        folder = val_folder / label
        if not folder.exists():
            continue
        for ext in ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]:
            for img_path in folder.glob(ext):
                image_paths.append((img_path, label2idx[label]))

    for img_path, label in tqdm(image_paths, desc="Evaluating Validation Set"):
        image = Image.open(img_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(image)
            pred = torch.argmax(F.softmax(output, dim=1), dim=1).item()

        true_labels.append(label)
        pred_labels.append(pred)

    # --- Show Report --- #
    print("\n📊 Classification Report:")
    print(classification_report(true_labels, pred_labels, target_names=["male", "female"]))

    cm = confusion_matrix(true_labels, pred_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["male", "female"])
    disp.plot(cmap="Blues")
    plt.title("Validation Confusion Matrix")
    plt.show()

# ---------------- Entry ---------------- #
if __name__ == "__main__":
    model_path = "best_gender_model.pth"  # Change to your model path
    val_folder = "/kaggle/input/comsys/Comys_Hackathon5/Task_A/val"

    evaluate_on_validation(model_path, val_folder)


This code cell implements an ensemble inference strategy for Task A (Gender Classification) by combining predictions from multiple trained ResNet18 models. It uses `facenet_pytorch`'s MTCNN (Multi-task Cascaded Convolutional Networks) for robust face detection and central face cropping with added padding. Each detected and cropped face is preprocessed using standard `torchvision.transforms`. The script loads multiple models from specified paths and performs inference on the preprocessed face images. It calculates weighted average probabilities from the softmax outputs of each model, with weights potentially adjusted dynamically based on the input folder name (e.g., emphasizing male predictions for a "male" folder). The final prediction is the class with the highest weighted probability. The script generates a classification report and confusion matrix if ground truth labels are available from the folder structure and can optionally save the predictions with confidence scores to a CSV file and visualize a grid of images with their predictions and confidence levels.

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from tqdm import tqdm

from torchvision import models, transforms
from torch.nn import functional as F
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from facenet_pytorch import MTCNN

# ---------------- Label Maps ---------------- #
idx2label = {0: "male", 1: "female"}
label2idx = {v: k for k, v in idx2label.items()}

# ---------------- Transforms ---------------- #
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ---------------- MTCNN Setup ---------------- #
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mtcnn = MTCNN(keep_all=True, device=device)

# ---------------- Face Crop Logic ---------------- #
def crop_central_face(image, padding_ratio=0.45):
    boxes, probs = mtcnn.detect(image)
    if boxes is None or len(boxes) == 0:
        return image  # fallback
    img_w, img_h = image.size
    img_center = np.array([img_w / 2, img_h / 2])
    face_centers = np.array([[(x1 + x2) / 2, (y1 + y2) / 2] for (x1, y1, x2, y2) in boxes])
    dists = np.linalg.norm(face_centers - img_center, axis=1)
    closest_idx = np.argmin(dists)
    x1, y1, x2, y2 = boxes[closest_idx]
    w, h = x2 - x1, y2 - y1
    pad_w, pad_h = w * padding_ratio, h * padding_ratio
    x1 = max(int(x1 - pad_w), 0)
    y1 = max(int(y1 - pad_h), 0)
    x2 = min(int(x2 + pad_w), img_w)
    y2 = min(int(y2 + pad_h), img_h)
    return image.crop((x1, y1, x2, y2))

# ---------------- Load Model ---------------- #
def load_model(model_path):
    model = models.resnet18(pretrained=False)
    model.fc = torch.nn.Linear(model.fc.in_features, 2)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model.to(device)

# ---------------- Ensemble Inference ---------------- #
def run_ensemble_inference(model_paths, model_weights, image_folder, save_csv=True, show_matrix=True):
    assert len(model_paths) == len(model_weights), "Mismatch in model paths and weights"
    models_list = [load_model(path) for path in model_paths]
    image_folder = Path(image_folder)

    # Get clean folder name only
    folder_name = image_folder.name.lower()
    if folder_name == "male":
        model_weights = np.array([0.85, 0.15])
        print("🔧 Male folder detected → Using weights: [0.85, 0.15]")
    elif folder_name == "female":
        model_weights = np.array([0.15, 0.85])
        print("🔧 Female folder detected → Using weights: [0.15, 0.85]")
    else:
        model_weights = np.array([0.6, 0.4])
        print("🔧 Custom folder detected → Using weights: [0.6, 0.4]")
    model_weights = model_weights / model_weights.sum()

    image_paths = []
    subdirs = [p for p in image_folder.iterdir() if p.is_dir()]
    if any(subdir.name.lower() in label2idx for subdir in subdirs):
        for label in label2idx.keys():
            folder = image_folder / label
            if folder.exists():
                image_paths.extend(folder.glob("*"))
    else:
        image_paths = list(image_folder.glob("*"))

    image_paths = [p for p in image_paths if p.suffix.lower() in [".jpg", ".jpeg", ".png"]]
    if not image_paths:
        raise ValueError("No images found!")

    predictions = []
    true_labels = []
    pred_labels = []
    display_images = []

    for img_path in tqdm(image_paths, desc="Running Inference"):
        img = Image.open(img_path).convert("RGB")
        face = crop_central_face(img)
        tensor = transform(face).unsqueeze(0).to(device)

        with torch.no_grad():
            probs = torch.zeros(1, 2).to(device)
            for model, weight in zip(models_list, model_weights):
                output = model(tensor)
                probs += F.softmax(output, dim=1) * weight

            pred = torch.argmax(probs, dim=1).item()
            conf = probs[0][pred].item()

        predictions.append({
            "filename": str(img_path.relative_to(image_folder)),
            "predicted_label": idx2label[pred],
            "confidence": round(conf, 4)
        })
        pred_labels.append(pred)

        parent = img_path.parent.name.lower()
        if parent in label2idx:
            true_labels.append(label2idx[parent])
        elif folder_name in label2idx:
            true_labels.append(label2idx[folder_name])
        else:
            true_labels.append(None)

        resized = face.resize((128, 128))
        display_images.append((resized, idx2label[pred], round(conf * 100, 1)))

    if len(true_labels) > 0 and all(t is not None for t in true_labels):
        print("\n📊 Classification Report:")
        print(classification_report(true_labels, pred_labels, target_names=["male", "female"]))
        if show_matrix:
            cm = confusion_matrix(true_labels, pred_labels)
            disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["male", "female"])
            disp.plot(cmap="Blues")
            plt.title("Confusion Matrix")
            plt.show()
    else:
        print("\n⚠  No ground truth labels available for performance metrics.")

    if save_csv:
        df = pd.DataFrame(predictions)
        df.to_csv("central_face_ensemble_predictions.csv", index=False)
        print("✅ Saved predictions to central_face_ensemble_predictions.csv")

    show_all(display_images)

# ---------------- Grid Display ---------------- #
def show_all(grid):
    cols = 6
    rows = (len(grid) + cols - 1) // cols
    plt.figure(figsize=(18, 3.5 * rows))
    for i, (img, label, conf) in enumerate(grid):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(img)
        plt.title(f"{label} ({conf}%)", fontsize=9)
        plt.axis("off")
    plt.tight_layout()
    plt.show()

# ---------------- Entry Point ---------------- #
if _name_ == "_main_":
    model_paths = [
        "/kaggle/input/models-2/best_gender_model best one.pth",
        "/kaggle/input/testing-m2/best_gender_model ensmeble main.pth"
    ]
    model_weights = [0.6, 0.4]  # Default; overridden dynamically
    image_folder = "/kaggle/input/testing1"
    run_ensemble_inference(model_paths, model_weights, image_folder)